In [1]:
import pandas as  pd
import os
import re

In [2]:
def generateCol(n):
    names = []
    for i in range(n):
        col_name = ''
        while i >= 0:
            col_name = chr(i % 26 + 65) + col_name
            i = i // 26 - 1
        names.append(col_name)
    return names


In [53]:
folderPath = "E:\\office\\masterFile\\FOLDER"
pattern = r'\bSrNo\b\s?|sr_no\b\s?|sr.no\b\s?|Sr.No\b\s?'
master = pd.read_excel("E:\\office\\masterFile\\Master datasheet.xlsx")
n = len(master.columns)
Colnames = generateCol(n)
masterColames= master.columns
master.columns = Colnames

for filename in os.listdir(folderPath):
    try:
        
        # reading the movement sheets
        Col_directions =pd.read_excel("E:\\office\\masterFile\\Master datasheet.xlsx",sheet_name=filename)
        # print(Col_directions)

        # reading the file in the folder 
        filePath = os.path.join(folderPath,filename)
        # print(filename)
        fileRead = pd.read_excel(filePath)
        df = pd.DataFrame()

        if 'Measurement' in fileRead.columns:
            fileRead[['Length','Width','Height']] =  fileRead['Measurement'].str.split('x',expand=True)
            fileRead.drop(columns=['Measurement'],inplace=True)

        n = len(fileRead.columns)
        Colnames = generateCol(n)
        fileRead.columns = Colnames
        # print(filename,fileRead)
           
        # taking the data till the data is not empty or NaH
        for index,row in fileRead.iterrows():
            # print(filename,row)
            # print(row['A'])
            # print(filename,fileRead,Col_directions)
          
            
            if (str(row['A']))== 'nan':
                break 
          
            for _ , movement_row in Col_directions.iterrows():
                # print(row[movement_row['From']])
                master.loc[index,movement_row['To']] = row[movement_row['From']]

              
                
            cleaned = master.T 
            df = pd.concat([df, cleaned.T], ignore_index=True)
        df.columns = masterColames
        # print(df)
        # df.to_excel("E:\\office\\masterFile\\master\\master.xlsx")
         
            
    except:
        continue
        
        
    

   Company   Shape   Size Colour Clarity Cut Polish Symmetry  Flour  Ratio  \
0      NaN  RADIANT  0.74    I +   VS1 -   -     EX       EX   NONE  1.399   
1      NaN  RADIANT  0.74    I +   VS1 -   -     EX       EX   NONE  1.399   
2      NaN  RADIANT   0.5    I +  VVS2 -   -     EX       VG   NONE  1.364   
3      NaN  RADIANT  0.74    I +   VS1 -   -     EX       EX   NONE  1.399   
4      NaN  RADIANT   0.5    I +  VVS2 -   -     EX       VG   NONE  1.364   
5      NaN  RADIANT   0.5    H +  VVS2 -   -     EX       EX   NONE  1.364   
6      NaN  RADIANT  0.74    I +   VS1 -   -     EX       EX   NONE  1.399   
7      NaN  RADIANT   0.5    I +  VVS2 -   -     EX       VG   NONE  1.364   
8      NaN  RADIANT   0.5    H +  VVS2 -   -     EX       EX   NONE  1.364   
9      NaN  RADIANT   0.7    H -  VVS2 -   -     EX       EX   NONE  1.488   
10     NaN  RADIANT  0.74    I +   VS1 -   -     EX       EX   NONE  1.399   
11     NaN  RADIANT   0.5    I +  VVS2 -   -     EX       VG   N

In [133]:
master = pd.read_excel("E:\\office\\masterFile\\Master datasheet.xlsx")
n = len(master.columns)

Colnames = generateCol(n)
master.columns = Colnames

In [ ]:
finestar = pd.read_excel("E:\\office\\masterFile\\Master datasheet.xlsx",sheet_name='Finestar')
finestar

In [173]:
excelDf = pd.read_excel("E:\\office\\masterFile\\FINESTAR_01_01_2025_03_15_33263.xlsx")

for index,rows in excelDf.iterrows(): 
    if (str(rows['SrNo']))!= 'nan':
        break 
    df = rows 
cleaned = pd.DataFrame(df).T

n = len(cleaned.columns)
Colnames = generateCol(n)
cleaned.columns = Colnames
for index,rows in cleaned.iterrows():
    for indexs,rowes in finestar.iterrows():
        master[rowes[2]] = cleaned[rowes[0]]
        
        # print(rowes[0],rowes[2])
        # master.loc[rowes[2], index] = cleaned.loc[rowes[0], index]
df = pd.DataFrame(master)
print(df)
        # master[rowes[2]][index] = cleaned[rowes[0]][index]
        # master[rowes[2]][index]
# master


     A   B     C  D    E  F   G   H    I     J  ...    S    T    U  \
0  NaN  RA  0.45  F  SI1  -  EX  VG  NON  1.45  ...  NaN  NaN  NaN   

            V    W    X    Y    Z                      AA   AB  
0  3114351802  NaN  DNA  CN1  SN1  Crystal, Twinning Wisp  NaN  

[1 rows x 28 columns]


C:\Users\91965\AppData\Local\Temp\ipykernel_15948\2184033324.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  master[rowes[2]] = cleaned[rowes[0]]


,A,B,C,D,E,F,G,H,I,J,...,S,T,U,V,W,X,Y,Z,AA,AB
